# Method 2: Decision Trees

We'll be applying the CART-algorithm (which uses a Gini impurity as a splitting criterion).
The CART-algorithm is the current standard for building classification trees, and is also as the default by scikit-learn. Other options included ID3 and C4.5.

CART is very similar to C4.5, though instead of building a tree with an undetermined amount of branches CART builds a binary tree. CART also uses the Gini impurity to determine what condition to slit the node on, instead of using regular information gain.

We will still calculate the information gain, though it will now be in the form of a probabilistic function based of of the Gini impurity.

Gini impurity is more commonly known as a method to show the disparity in wealth between two classes but it can be generalised a bit to show the disparity between the amount of values in one class and all others. So the more classes that arrive at a note, the higher the impurity.

CART decides the root node by exhaustively trying all possible splitting conditions for their Gini impurity, which takes a little while because our data is sporadic and has floats for data points. Meaning it has to test every none duplicate value in afeature that's being considered for a question.

Step 1: Import data

In [1]:
import pandas as pd
import math
import numpy as np
train = pd.read_csv("Final_project_DATA/df_train.csv")
test = pd.read_csv("Final_project_DATA/df_test.csv")

test.iloc[:]['type'].to_numpy()

array([1, 1, 1, 7, 6, 7, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 7, 2, 2, 7, 5, 7,
       3, 2, 3, 2, 6, 2, 1, 2, 2, 1, 3, 1, 7, 1, 5, 7, 1, 2, 1, 1, 7, 1,
       5, 2, 5, 2, 6, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 7, 1, 2, 3, 2, 2])

Find the occurences of every label in the data set.

In [2]:
def count_occ_labels(data):
    # Counts the amount of occurences in a pandas series.
    out = {}
    labels = data['type'].to_numpy()
    val, occ = np.unique(labels, return_counts=True)
    for A, B in zip(val, occ):
        out[A] = B
    
    return out

In [3]:
count_occ_labels(train)

{1: 49, 2: 53, 3: 12, 5: 9, 6: 6, 7: 20}

Step 2: Create the split condition object class

The split condition should hold the feature(column number) and the threshold/value that will be used as the condition.
It also needs a method to compare the question value against the value of the passed sample.

In [4]:
class SplitCondition:
    def __init__(self, feature, threshold):
        self.feature = feature
        self.threshold = threshold
        
    def compare(self, sample):
        # Take sample value for feature, compare with threshold
        s_value = sample[self.feature]
        if isinstance(s_value,float) or isinstance(s_value,int):
            return s_value >= self.threshold
        else:
            return s_value == self.threshold

Next we need a method to split the dataset based on the questions.
So we need to divide the samples in X into true and false samples.
Or more concretely, into the values that will proceed to it's left and right child nodes.

In [5]:
def split(data, SplitCondition):
    # Bad practise to append to DataFrames, so work with lists
    trueSamples = []
    falseSamples = []
    for i in range(data.shape[0]):
        if SplitCondition.compare(data.iloc[i]):
            trueSamples.append(data.iloc[i])
        else:
            falseSamples.append(data.iloc[i])
    return pd.DataFrame(trueSamples), pd.DataFrame(falseSamples)

Now we need a function to assign a value to the quality of the node, a means to compare a questions to others. 
CART uses the Gini Impurity for this purpose.

In [6]:
def gini(samples):
    cnt = count_occ_labels(samples)
    impurity = 1
    for i in cnt:
        probability = cnt[i]/float(len(samples))
        impurity -= probability**2
    return impurity

To use Gini properly, we need to minimise the information gain compared to the node's child nodes. We need to know the uncertainty of the node and the impurity of the two new child nodes which will be introduced based on the question we're considering.

In [7]:
def InformationGain(true, false, previous):
    p = float(len(true)) / (len(true) + len(false))
    return previous - p * gini(true) - (1 - p) * gini(false)

We find the best vondition by trying every possible question and see what works best.

In [8]:
def BestCondition(samples):
    mostGains = 0 
    bestCondition = None 
    previous = gini(samples) # needed for IG 
    n = samples.shape[1] - 1 # number of features
    
    for i in range(n):
        uniques = samples.iloc[:,i].unique()
        
        for j in uniques:
            condition = SplitCondition(samples.columns[i], j) 
            trueSamples, falseSamples = split(samples, condition)
            
            if len(trueSamples.index) == 0 or len(falseSamples.index) == 0: 
                continue
                
            gains = InformationGain(trueSamples, falseSamples, previous)
            if gains > mostGains:
                mostGains, bestCondition = gains, condition
                
    return mostGains, bestCondition

We only need the nodes and leafs and a method to build the tree at this point.

In [9]:
class Leaf:
    def __init__(self, samples):
        self.pred = count_occ_labels(samples)

In [10]:
class Node:
    def __init__(self, condition, left, right):
        self.condition = condition
        self.left = left
        self.right = right

Treebuilder just recursively creates splits in the data until there's no more information gain, at which point it puts a leaf down.

In [11]:
def Treebuilder(samples,height=10000,nodes=1):
    gain, condition = BestCondition(samples)
    
    if gain == 0 or math.floor(math.log(nodes,2)) == height:
        print(nodes)
        return Leaf(samples)
    
    trueSamples, falseSamples = split(samples, condition)
    nodes += 2
    
    left = Treebuilder(trueSamples, height, nodes)
    right = Treebuilder(falseSamples, height, nodes)
    
    return Node(condition, left, right)

In [16]:
tree = Treebuilder(train,4)

7
7
7
7
9
15
17
17
15
15
11
9
9
9
11
11
11
11
15
17
17
17
17
17
17
15
17
17
15
15


In [13]:
print(math.floor(math.log(9,2)))

3


Lastly we need to classify based on the just now generated nodes and their branches.

In [17]:
def classify(sample, Node):
    if isinstance(Node, Leaf):
        return Node.pred
    
    if Node.condition.compare(sample):
        return classify(sample, Node.left)
    else:
        return classify(sample, Node.right)

In [18]:
from sklearn.metrics import confusion_matrix
# Slightly more in-depth results
recall = dict.fromkeys([1,2,3,5,6,7])
precision = dict.fromkeys([1,2,3,5,6,7])
f1 = dict.fromkeys([1,2,3,5,6,7])
accuracies = []

# Confusion matrix 
y_true = test.iloc[:]['type'].to_numpy()
y_pred = np.zeros(shape=y_true.shape)
for j in recall:
    true_pos, true_neg, false_pos, false_neg = 0,0,0,0
    for i in range(len(test.index)):
        label = test.iloc[i]['type']
        pred = list(classify(test.iloc[i],tree).keys())[0]
        y_pred[i] = pred
        if j == label:
            if label == pred:
                true_pos += 1
            else:
                false_neg += 1
        else:
            if j != pred:
                true_neg += 1
            else:
                false_pos += 1
    accuracies.append((true_pos + true_neg)/(true_pos + true_neg + false_neg + false_pos))
    print("Class {}".format(j))
    print("")
    print(" True Pos  : {}     False Pos : {}".format(true_pos,false_pos))
    print("")
    print(" False Neg : {}     True Neg  : {}".format(false_neg, true_neg))
    print("")
    print("")
    
    if true_pos + false_neg == 0:  # to fix scenarios where they equal 0
        false_neg += 1
    if true_pos + false_pos == 0:  # to fix scenarios where they equal 0
        false_pos += 1

    recall[j] = true_pos / (true_pos + false_neg)

    precision[j] = true_pos / (true_pos + false_pos)
    print("Type = ", j)
    rec = recall[j]
    print("Recall = ", rec)
    pre = precision[j]
    print("Precision = ", pre)
    print("")
    print()
        
    if pre == 0:
        f1[j] = 0
    else:
        f1[j]   = 2 * ((pre * rec) / (pre + rec))

accuracy = sum(accuracies) / len(accuracies)
macro_f1 = sum([f1[k] for k in [1,2,3,5,6,7]]) / 6
recall_fin = sum([recall[k] for k in [1,2,3,5,6,7]]) / 6
precision_fin = sum([precision[k] for k in [1,2,3,5,6,7]]) / 6

print("")
print("")
print("Accuracy            : {}".format(round(accuracy , 4)))
print("Precision           : {}".format(round(precision_fin,4)))
print("Recall              : {}".format(round(recall_fin,4)))
print("Macro F1            : {}".format(round(macro_f1,4)))
print("")
#print(y_true)
#print(y_pred)
print(confusion_matrix(y_true, y_pred,labels=[1,2,3,5,6,7]))

Class 1

 True Pos  : 17     False Pos : 12

 False Neg : 4     True Neg  : 32


Type =  1
Recall =  0.8095238095238095
Precision =  0.5862068965517241


Class 2

 True Pos  : 13     False Pos : 2

 False Neg : 10     True Neg  : 40


Type =  2
Recall =  0.5652173913043478
Precision =  0.8666666666666667


Class 3

 True Pos  : 0     False Pos : 3

 False Neg : 5     True Neg  : 57


Type =  3
Recall =  0.0
Precision =  0.0


Class 5

 True Pos  : 4     False Pos : 3

 False Neg : 0     True Neg  : 58


Type =  5
Recall =  1.0
Precision =  0.5714285714285714


Class 6

 True Pos  : 2     False Pos : 2

 False Neg : 1     True Neg  : 60


Type =  6
Recall =  0.6666666666666666
Precision =  0.5


Class 7

 True Pos  : 7     False Pos : 0

 False Neg : 2     True Neg  : 56


Type =  7
Recall =  0.7777777777777778
Precision =  1.0




Accuracy            : 0.8872
Precision           : 0.5874
Recall              : 0.6365
Macro F1            : 0.5897

[[17  2  2  0  0  0]
 [ 6 13  0  3  1  0